In [1]:
import glob
import os
import librosa
import numpy as np
import ntpath
from pathlib import Path
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
def check_data(parent_dir, sub_dirs, file_ext="*.wav"):
    min_length = 0
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
                X, s = librosa.load(fn)
                if min_length == 0:
                    min_length = len(X)
                else:
                    if min_length > len(X):
                        min_length = len(X)
                        print(min_length)
            except Exception as e:
                print('cannot open', e)
                continue
    return min_length

In [5]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    X = X[0:5645]
#     X = librosa.util.pad_center(X, 30*sample_rate)
    stft = np.abs(librosa.stft(X))  
#     mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40), axis=1)
    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T
    print(mfcc.shape)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate).T  
    print(chroma.shape)
    mel = librosa.feature.melspectrogram(X, sr=sample_rate).T 
    print(mel.shape)
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T  
    print(contrast.shape)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T  
    print(tonnetz.shape)
    return mfcc, chroma, mel, contrast, tonnetz

In [3]:
def parse_audio_files(parent_dir, file_ext="**/*.wav"):
    features, labels = [], []
    path = Path(main_dir).glob(file_ext)
    wavs = [str(wavf) for wavf in path if wavf.is_file()]
    for wav in wavs:
        for fn in glob.glob(wav):
            try:
                print('process..', fn)
                extracted = extract_feature(fn)
            except Exception as e:
                print('cannot open', e)
                continue
            ext_features = np.hstack(extracted)
            features.append(ext_features)
            filename = ntpath.basename(fn)
            labelFile = 0
            if (filename.split('-')[0] == 'pos') :
                labelFile = 1
            labels.append(labelFile)  
    return np.array(features), np.array(labels, dtype=np.int32)

In [6]:
main_dir = "D:/Mata Kuliah/Tesis/Coswara-Data/Copy/covid"
# min_length = check_data(main_dir, sub_dir)
features, labels = parse_audio_files(main_dir)

scaler = StandardScaler()
features = scaler.fit_transform(features)

print("done")
np.save("X_mfccmean_coswaranew", features)
np.save("Y_mfccmean_coswaranew", labels)

process.. D:\Mata Kuliah\Tesis\Coswara-Data\Copy\covid\heavy\neg\neg-01OCEf1yB4czsq8ygRoT51s96Ba2-cough-heavy.wav
(12, 40)
(12, 12)
(12, 128)
(12, 7)
(12, 6)
process.. D:\Mata Kuliah\Tesis\Coswara-Data\Copy\covid\heavy\neg\neg-05bieNLXPuaIWEVaX81EkbbjVrh1-cough-heavy.wav
(12, 40)
(12, 12)
(12, 128)
(12, 7)
(12, 6)
process.. D:\Mata Kuliah\Tesis\Coswara-Data\Copy\covid\heavy\neg\neg-0bcMNFt3dlPlUPTyCO8DVAlbTUC3-cough-heavy.wav
(12, 40)
(12, 12)
(12, 128)
(12, 7)
(12, 6)
process.. D:\Mata Kuliah\Tesis\Coswara-Data\Copy\covid\heavy\neg\neg-0c00ybTdnIRUTXeg20bZjJOzhKv1-cough-heavy.wav


KeyboardInterrupt: 